# M2 tangent Actuator force limit

This Jupyter notebook is to run the M2 test for the software limit for the tangent actuators
A tangent actuator will be issued with a force over the software limit and check that the command will be rejected by the CSCejected by the CSC. This script shall be run with the M2 cell standing vertically (TMA pointing at the horizon).

## Import Modules

In [ ]:
import asyncio
import numpy as np
from datetime import datetime
from lsst.ts import salobj
from lsst.ts.m2com import NUM_ACTUATOR, NUM_TANGENT_LINK

## Declaration of User-defined Functions

In [ ]:
async def injectForce_tangent_actuator(csc, actuators, force, sleep_time=5):
    """inject a force to the axial actuator.
    
    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the M2 CSC.
    actuators : list of actuators.
    force : list of forces
        Force to apply (Newton).
    sleep_time : float, optional
        Sleep time. (the default is 5.0)
    """

    forces = [0.] * NUM_TANGENT_LINK
    
    for idx in range(len(actuators)):
        print(f"idx: {idx}.")
        index = actuators[idx]
        forces[index] = force[idx]
        print(f"Apply the force: {force[index]} N. to actuator: {index}")
    await csc.cmd_applyForces.set_start(tangent=forces)
    await asyncio.sleep(sleep_time)


## Prepare the M2 CSC and put to Enabled state

In [ ]:
domain = salobj.Domain()
m2 = salobj.Remote(domain, "MTM2")
await m2.start_task
await m2.cmd_setLogLevel.set_start(level=10)

In [ ]:
# get status

state = m2.evt_summaryState.get()
print(state)

In [ ]:
# Standby  -->  Disable
await m2.cmd_start.set_start(timeout=60)

In [ ]:
# Disable  -->  Enabled
await m2.cmd_enable.set_start(timeout=200)

In [ ]:
# Enabled  -->  Disable
await m2.cmd_disable.set_start(timeout=30)

In [ ]:
#Disable  -->  Standby
await m2.cmd_standby.set_start(timeout=30)

In [ ]:
#Fault --> Standby
await m2.cmd_standby.set_start(timeout=30)

## Do the force actuator limit Test

In [ ]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

In [ ]:
# set actuators number
actuators = [0,1,2,3,4,5]
n_act = NUM_TANGENT_LINK


# set tangent actuator forces
f_Delta = np.array(f_Delta, float)
f_Delta[1] = -300.
f_Delta[4] = +300.
forces = f_Delta

In [ ]:
# reset applied forces to tangent links

for idx in range(n_act):
    force = forces[idx]
await m2.cmd_resetForceOffsets.set_start()
await asyncio.sleep(5)

In [ ]:
#apply forces to the diametrally opposed tangent links to keep minimized the force momentum

for idx in range(n_act):
    force = forces[idx]
    
await injectForce_tangent_actuator(m2, actuators, f_Delta, sleep_time=5)
await asyncio.sleep(5)

In [ ]:
# reset applied forces to tangent links

for idx in range(n_act):
    force = forces[idx]
await m2.cmd_resetForceOffsets.set_start()
await asyncio.sleep(5)